In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Processed/grouped_augmented_trips.csv', index_col=0)

In [ ]:
df_members, df_casuals = df[df['User Type'] == 'Member'], df[df['User Type'] == 'Casual']

In [49]:
df_members['Start Time'] = pd.to_datetime(df_members['Start Time'])
df_members['End Time'] = pd.to_datetime(df_members['End Time'])

/tmp/ipykernel_2370904/448852404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_members['Start Time'] = pd.to_datetime(df_members['Start Time'])
/tmp/ipykernel_2370904/448852404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_members['End Time'] = pd.to_datetime(df_members['End Time'])


In [50]:
start_date = min(df_members['Start Time'].min().floor('d'), df_members['End Time'].min().floor('d'))
end_date = max(df_members['Start Time'].max().ceil('d'), df_members['End Time'].max().ceil('d'))

## yearly

In [17]:
from datetime import date
from dateutil.relativedelta import *

In [36]:
start_year = date(start_date.year, 1, 1)
years_begin = [start_year + relativedelta(years=+i) for i in range(end_date.year - start_date.year + 1)]
years_end = [d + relativedelta(years=+1) for d in years_begin]

In [38]:
from datetime import date
from dateutil.relativedelta import * 
from tqdm.notebook import tqdm

years_interval = list(zip(years_begin, years_end))

for year in tqdm(years_interval):

    year_begin, year_end = year
    
    year_df = df_members[(df_members['Start Time'] >= pd.to_datetime(year_begin)) & (df_members['End Time'] < pd.to_datetime(year_end))]

    year_df.to_csv(f'Processed/casuals/{year_begin.year}/trips-{year_begin.year}.csv')

    # G, _ = func2(year_df)
    
    # nx.write_gml(G, f"Processed/years/month-{month}.gml")


  0%|          | 0/8 [00:00<?, ?it/s]

## monthly

In [39]:
start_month = start_date.replace(day=1)

while start_month < end_date:

    end_month = start_month + relativedelta(months=+1)

    month_df = df_members[(df_members['Start Time'] >= pd.to_datetime(start_month)) & (df_members['End Time'] < pd.to_datetime(end_month))]

    month_df.to_csv(f'Processed/casuals/{start_month.year}/months/trips-{start_month.year}-{start_month.month}.csv')

    start_month = end_month

## weekly

In [23]:
# start_week = start_date.replace(day=1)

# while start_week < end_date:

#     end_week = start_week + relativedelta(weeks=+1)

#     week_df = df_members[(df_members['Start Time'] >= pd.to_datetime(start_week)) & (df_members['End Time'] < pd.to_datetime(end_week))]

#     year, week, _ = end_week.isocalendar()

#     week_df.to_csv(f'Processed/casuals/{year}/weeks/trips-{year}-{week}.csv')

#     start_week = end_week

## temperature

In [51]:
temp_edges = np.linspace(df['temperature'].min(), df['temperature'].max() + 0.01, 7)
temp_intervals = list(zip(temp_edges[:-1], temp_edges[1:]))

In [52]:
start_year = date(start_date.year, 1, 1)
years_begin = [start_year + relativedelta(years=+i) for i in range(end_date.year - start_date.year + 1)]
years_end = [d + relativedelta(years=+1) for d in years_begin]

In [53]:
from datetime import date
from dateutil.relativedelta import * 
from tqdm.notebook import tqdm

years_interval = list(zip(years_begin, years_end))

for year in tqdm(years_interval):

    year_begin, year_end = year
    
    year_df = df_members[(df_members['Start Time'] >= pd.to_datetime(year_begin)) & (df_members['End Time'] < pd.to_datetime(year_end))]

    for i, t_interval in enumerate(temp_intervals):

        year_temp_df = year_df[(year_df['temperature'] >= t_interval[0]) & (year_df['temperature'] < t_interval[1])]
        
        year_temp_df.to_csv(f'Processed/members/{year_begin.year}/temp/trips-temp-{i}.csv')

    # G, _ = func2(year_df)
    
    # nx.write_gml(G, f"Processed/years/month-{month}.gml")


  0%|          | 0/8 [00:00<?, ?it/s]

## csv to gml

In [68]:
import glob
from tqdm.notebook import tqdm

temp_csvs = glob.glob('Processed/members/**/temp/*.csv', recursive=True)

for temp_csv in tqdm(temp_csvs):

    try:

        new_name = temp_csv.replace('trips-temp', 'network-temp').replace('csv', 'gml')
        temp_df = pd.read_csv(temp_csv)
    
        G, _ = func2(temp_df)
        nx.write_gml(G, new_name)

    except Exception as e:

        print(temp_csv)
        print(len(temp_df))
        

  0%|          | 0/48 [00:00<?, ?it/s]

Processed/members/2024/temp/trips-temp-0.csv
0


In [69]:
import glob
from tqdm.notebook import tqdm

temp_csvs = glob.glob('Processed/casuals/**/temp/*.csv', recursive=True)

for temp_csv in tqdm(temp_csvs):

    try:

        new_name = temp_csv.replace('trips-temp', 'network-temp').replace('csv', 'gml')
        temp_df = pd.read_csv(temp_csv)
    
        G, _ = func2(temp_df)
        nx.write_gml(G, new_name)

    except Exception as e:

        print(temp_csv)
        print(len(temp_df))
        

  0%|          | 0/48 [00:00<?, ?it/s]

In [70]:
import glob
from tqdm.notebook import tqdm

month_csvs = glob.glob('Processed/members/**/months/*.csv', recursive=True)

for month_csv in tqdm(month_csvs):

    try:

        new_name = month_csv.replace('trips-', 'network-').replace('csv', 'gml')
        month_df = pd.read_csv(month_csv)
    
        G, _ = func2(month_df)
        nx.write_gml(G, new_name)

    except Exception as e:

        print(month_csv)
        print(len(month_df))
        

  0%|          | 0/93 [00:00<?, ?it/s]

Processed/members/2023/months/trips-2023-9.csv
0
Processed/members/2023/months/trips-2023-10.csv
0
Processed/members/2023/months/trips-2023-11.csv
0
Processed/members/2023/months/trips-2023-12.csv
0
Processed/members/2024/months/trips-2024-1.csv
0


In [71]:
import glob
from tqdm.notebook import tqdm

month_csvs = glob.glob('Processed/casuals/**/months/*.csv', recursive=True)

for month_csv in tqdm(month_csvs):

    try:

        new_name = month_csv.replace('trips-', 'network-').replace('csv', 'gml')
        month_df = pd.read_csv(month_csv)
    
        G, _ = func2(month_df)
        nx.write_gml(G, new_name)

    except Exception as e:

        print(month_csv)
        print(len(month_df))
        

  0%|          | 0/93 [00:00<?, ?it/s]

## average months

In [75]:
from collections import Counter

for month in range(1, 13):

    files = glob.glob(f'Processed/casuals/*/months/*network*-{month}.gml', recursive=True)
    g_count = len(files)
    
    G_average = nx.DiGraph()
    
    for weight_name in ['trip_count_weights', 'duration_weights', 'tpd_weights']:
    
        c = Counter()
        
        for filename in files:
        
            G = nx.read_gml(filename)
        
            G_weights = dict(map(lambda x: ((x[0], x[1]), x[2]), G.edges.data(weight_name)))
    
            c.update(G_weights)
    
        edge_list = [(*x[0], {weight_name: x[1] / g_count}) for x in c.items()]
    
        G_average.add_edges_from(edge_list)

    nx.write_gml(G_average, f'Processed/casuals/average/months/network-{month}.gml')

    N, E = G_average.number_of_nodes(), G_average.number_of_edges()
    print(f"density of average networks of month {month} is: {E / (N * (N - 1))}")

density of average networks of month 1 is: 0.20750015690704826
density of average networks of month 2 is: 0.14596774193548387
density of average networks of month 3 is: 0.17672583051912355
density of average networks of month 4 is: 0.2265123901556893
density of average networks of month 5 is: 0.300997769072672
density of average networks of month 6 is: 0.3147211474617692
density of average networks of month 7 is: 0.3288832825256666
density of average networks of month 8 is: 0.3285228519706723
density of average networks of month 9 is: 0.3244558820946124
density of average networks of month 10 is: 0.387579348947747
density of average networks of month 11 is: 0.31912696918345573
density of average networks of month 12 is: 0.23581246469591413


In [76]:
from collections import Counter

for month in range(1, 13):

    files = glob.glob(f'Processed/members/*/months/*network*-{month}.gml', recursive=True)
    g_count = len(files)
    
    G_average = nx.DiGraph()
    
    for weight_name in ['trip_count_weights', 'duration_weights', 'tpd_weights']:
    
        c = Counter()
        
        for filename in files:
        
            G = nx.read_gml(filename)
        
            G_weights = dict(map(lambda x: ((x[0], x[1]), x[2]), G.edges.data(weight_name)))
    
            c.update(G_weights)
    
        edge_list = [(*x[0], {weight_name: x[1] / g_count}) for x in c.items()]
    
        G_average.add_edges_from(edge_list)

    nx.write_gml(G_average, f'Processed/members/average/months/network-{month}.gml')

    N, E = G_average.number_of_nodes(), G_average.number_of_edges()
    print(f"density of average networks of month {month} is: {E / (N * (N - 1))}")

density of average networks of month 1 is: 0.20265102431685872
density of average networks of month 2 is: 0.14291822186559028
density of average networks of month 3 is: 0.17872672313461788
density of average networks of month 4 is: 0.1961353687637289
density of average networks of month 5 is: 0.22105021179095252
density of average networks of month 6 is: 0.23279849449037895
density of average networks of month 7 is: 0.24968068382786401
density of average networks of month 8 is: 0.2521186278780098
density of average networks of month 9 is: 0.25305859883809206
density of average networks of month 10 is: 0.3290466327747442
density of average networks of month 11 is: 0.2748164187535304
density of average networks of month 12 is: 0.23020146864997176


## average temp

In [4]:
from collections import Counter
import glob
import networkx as nx

for temp_int in range(6):

    files = glob.glob(f'Processed/members/*/temp/*network*-{temp_int}.gml', recursive=True)
    g_count = len(files)
    
    G_average = nx.DiGraph()
    
    for weight_name in ['trip_count_weights', 'duration_weights', 'tpd_weights']:
    
        c = Counter()
        
        for filename in files:
        
            G = nx.read_gml(filename)
        
            G_weights = dict(map(lambda x: ((x[0], x[1]), x[2]), G.edges.data(weight_name)))
    
            c.update(G_weights)
    
        edge_list = [(*x[0], {weight_name: x[1] / g_count}) for x in c.items()]
    
        G_average.add_edges_from(edge_list)

    nx.write_gml(G_average, f'Processed/members/average/temp/network-temp-{temp_int}.gml')

    N, E = G_average.number_of_nodes(), G_average.number_of_edges()
    print(f"density of average networks of temp interval {temp_int} is: {E / (N * (N - 1))}")

density of average networks of temp interval 0 is: 0.0407578836456687
density of average networks of temp interval 1 is: 0.1476444921922856
density of average networks of temp interval 2 is: 0.23508589971753077
density of average networks of temp interval 3 is: 0.2608874056242477
density of average networks of temp interval 4 is: 0.3113203484177999
density of average networks of temp interval 5 is: 0.2574102878070843


In [ ]:
from collections import Counter
import glob
import networkx as nx

for temp_int in range(6):

    files = glob.glob(f'Processed/casuals/*/temp/*network*-{temp_int}.gml', recursive=True)
    g_count = len(files)
    
    G_average = nx.DiGraph()
    
    for weight_name in ['trip_count_weights', 'duration_weights', 'tpd_weights']:
    
        c = Counter()
        
        for filename in files:
        
            G = nx.read_gml(filename)
        
            G_weights = dict(map(lambda x: ((x[0], x[1]), x[2]), G.edges.data(weight_name)))
    
            c.update(G_weights)
    
        edge_list = [(*x[0], {weight_name: x[1] / g_count}) for x in c.items()]
    
        G_average.add_edges_from(edge_list)

    nx.write_gml(G_average, f'Processed/casuals/average/temp/network-temp-{temp_int}.gml')

    N, E = G_average.number_of_nodes(), G_average.number_of_edges()
    print(f"density of average networks of temp interval {temp_int} is: {E / (N * (N - 1))}")

density of average networks of temp interval 0 is: 0.01874932053611642
density of average networks of temp interval 1 is: 0.10326321467098167
density of average networks of temp interval 2 is: 0.25047985664504063
density of average networks of temp interval 3 is: 0.3302109075192661
density of average networks of temp interval 4 is: 0.4437219504903341


## see self loops

In [50]:
import glob

files = glob.glob('Processed/**/average/months/*.gml', recursive=True)

In [49]:
import networkx as nx

def find_selfloops(filename):
    try:
        G = nx.read_gml(filename)
        self_loops = list(nx.selfloop_edges(G))
        isolates = list(nx.isolates(G))
        if len(self_loops) != 0 or len(isolates) != 0:
            return filename
        return None
    except Exception as e:
        return e

import multiprocessing as mp

with mp.Pool() as pool:

    results = pool.map(find_selfloops, files, chunksize=1)


In [51]:
from tqdm.notebook import tqdm

for filename in tqdm(files):
    try:
        G = nx.read_gml(filename)
        G.remove_edges_from(list(nx.selfloop_edges(G)))
        G.remove_nodes_from(list(nx.isolates(G)))
        nx.write_gml(G, filename)
    except Exception as e:
        print(filename)

  0%|          | 0/36 [00:00<?, ?it/s]

## util

In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

def get_tempdf(df):
    temp1_df = df[["Start Station Id", "Start Station Name"]].rename({'Start Station Id':'Id', 'Start Station Name': 'Name'}, axis=1)
    temp2_df = df[["End Station Id", "End Station Name"]].rename({'End Station Id':'Id', 'End Station Name': 'Name'}, axis=1)
    temp_df = pd.concat([temp1_df, temp2_df]).drop_duplicates()
    return temp_df
    
def func2(df):
    
    whole_trips_df = df[['Start Time', 'End Time', 'Trip Id', 'Trip Duration', 'Start Station Id', 'End Station Id']]
    whole_trips_df = whole_trips_df.astype({'Start Time': 'datetime64[ns]', 'End Time': 'datetime64[ns]'})

    temp_df = whole_trips_df[['Start Time', 'End Time', 'Start Station Id', 'End Station Id', 'Trip Id']].groupby(['Start Station Id', 'End Station Id']).agg({'Trip Id': 'count', 'Start Time': 'min', 'End Time': 'max'}).reset_index().rename({'Trip Id': 'Trip Count'}, axis=1)
    temp_df['Link Duration'] = temp_df.apply(lambda x: (x['End Time'] - x['Start Time']).ceil('d'), axis=1).dt.days    #.astype('timedelta64[D]')
    temp_df['Avg Duration'] = whole_trips_df[['Start Station Id', 'End Station Id', 'Trip Duration']].groupby(['Start Station Id', 'End Station Id']).mean().reset_index()['Trip Duration'] # why using trip duration and not link duration
    temp_df['duration_weights'] = temp_df['Avg Duration'].apply(lambda x: 1 / x)

    temp_df['tpd_weights'] = temp_df.apply(lambda x: x['Trip Count'] / x['Link Duration'], axis=1)
    temp_df = temp_df.rename({'Trip Count': 'trip_count_weights'}, axis=1).drop(['Start Time', 'End Time', 'Link Duration', 'Avg Duration'], axis=1)

    nnp = np.percentile(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']], 99, axis=0)
    temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']] = np.clip(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']], None, nnp)

    min_max_scaler = MinMaxScaler((1, 10))
    x_scaled = min_max_scaler.fit_transform(temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']])
    temp_df[['trip_count_weights', 'duration_weights', 'tpd_weights']] = x_scaled

    G = nx.from_pandas_edgelist(temp_df, source="Start Station Id", target="End Station Id", edge_attr=['trip_count_weights', 'duration_weights', 'tpd_weights'], create_using=nx.DiGraph)

    node_labels = {d['Id']: d['Name'] for d in get_tempdf(df)[['Id', 'Name']].to_dict('records')}
    nx.set_node_attributes(G, node_labels, name='Name')

    return G, temp_df